In [45]:
import pandas as pd
import numpy as np

In [ ]:
df2 = pd.read_csv(r"X:\Model Runs\2021\Data Preparation\07 TSD Data\TSD Extract.All Roads - RA07 .2020 Network Survey.2021-04-22.09-12.331903072.csv")
df2

In [2]:
colmap = {
    "ROAD_NO":"road",
    "START_SLK":"slk_from",
    "END_SLK":"slk_to",
    "CWAY":"cwy",
    "D0":"deflection",
    "DIRN":"dirn"
}

df2 = df2.loc[:,colmap.keys()].rename(columns=colmap)
df2

,road,slk_from,slk_to,cwy,deflection,dirn
0,H001,0.00,0.01,L,179.37,L
1,H001,0.01,0.02,L,177.12,L
2,H001,0.02,0.03,L,179.06,L
3,H001,0.03,0.04,L,212.65,L
4,H001,0.04,0.05,L,175.35,L
...,...,...,...,...,...,...
282820,M045,12.16,12.17,S,444.11,R
282821,M045,12.17,12.18,S,394.53,R
282822,M045,12.18,12.19,S,355.79,R
282823,M045,12.19,12.20,S,405.11,R


In [3]:
df2 = df2[
      (df2["road"] == "H001")
    & (df2["slk_to"]<100)
]
df2

,road,slk_from,slk_to,cwy,deflection,dirn
0,H001,0.00,0.01,L,179.37,L
1,H001,0.01,0.02,L,177.12,L
2,H001,0.02,0.03,L,179.06,L
3,H001,0.03,0.04,L,212.65,L
4,H001,0.04,0.05,L,175.35,L
...,...,...,...,...,...,...
17064,H001,36.04,36.05,S,217.21,R1
17065,H001,36.05,36.06,S,255.90,R1
17066,H001,36.06,36.07,S,311.99,R1
17067,H001,36.07,36.08,S,299.76,R1


In [3]:
df2.to_csv("./df2.csv", index=False)

In [26]:
data = df2
by_category = ["road","cwy","dirn"]
combo_lookup = (
    data
    .loc[:, by_category]
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
    .set_index(by_category)
    .iloc[:,0]
    .rename("category.id")
)
combo_lookup

road  cwy  dirn
H001  L    L         0
      S    L         1
      L    L1        2
      S    L1        3
      R    R         4
                  ... 
M010  S    R       205
M026  S    L       206
           R       207
M045  S    L       208
           R       209
Name: category.id, Length: 210, dtype: int64

In [27]:

category_id = (
    data
    #.loc[:,by_category]
    #.set_index(by_category)
    .join(combo_lookup, on=by_category)
    .loc[:, "category.id"]
)

category_id

0           0
1           0
2           0
3           0
4           0
         ... 
282820    209
282821    209
282822    209
282823    209
282824    209
Name: category.id, Length: 282825, dtype: int64

In [28]:
data["xxx"]=1
data["xxx"].groupby(data.loc[:, by_category]).cumsum()

ValueError: Grouper for '<class 'pandas.core.frame.DataFrame'>' not 1-dimensional

In [38]:
data.groupby(by_category).ngroup()

0           0
1           0
2           0
3           0
4           0
         ... 
282820    209
282821    209
282822    209
282823    209
282824    209
Length: 282825, dtype: int64

In [31]:
by_category

['road', 'cwy', 'dirn']

In [42]:
import pandas


def segment_by_category_and_slk_breaks(
    data:pandas.DataFrame,
    categories:list[str]=["road", "cwy"],
    measure_from:str="slk_from",
    measure_to:str="slk_to",
    ):
    
    data = data.sort_values(by=[*categories, measure_from])

    for index, (group_index, group) in enumerate(data.groupby(categories)):
        group["xxx"] = index 
    return data
    

In [43]:
segment_by_category_and_slk_breaks(df2)

,road,slk_from,slk_to,cwy,deflection,dirn,xxx
0,H001,0.00,0.01,L,179.37,L,1
5052,H001,0.00,0.01,L,347.58,L1,1
1,H001,0.01,0.02,L,177.12,L,1
5053,H001,0.01,0.02,L,296.61,L1,1
2,H001,0.02,0.03,L,179.06,L,1
...,...,...,...,...,...,...,...
282822,M045,12.18,12.19,S,355.79,R,1
281679,M045,12.19,12.20,S,423.47,L,1
282823,M045,12.19,12.20,S,405.11,R,1
281680,M045,12.20,12.21,S,378.95,L,1


In [48]:

np.isclose(0.001, 0.0010002)

False